# Wormgear Calculator on Google Colab

This notebook demonstrates using the wormgear calculator on Google Colab.

**Note:** Due to dependency conflicts with Colab's pre-installed IPython, we install wormgear without its geometry dependencies. This means:
- ✅ Calculator works (design calculations, validation, JSON export)
- ❌ Geometry generation doesn't work (STEP file creation)

To generate STEP files from the JSON output, you can:
1. Use [wormgear.studio](https://wormgear.studio) (web interface)
2. Install wormgear locally: `pip install wormgear` and run `wormgear-geometry design.json`

In [ ]:
# Install wormgear without geometry dependencies (avoids IPython conflict)
!pip install wormgear --no-deps

In [ ]:
# Import the calculator
from wormgear.calculator import (
    design_from_module,
    design_from_centre_distance,
    design_from_envelope,
    validate_design
)
from wormgear.calculator.output import to_json
import json

## Example 1: Design from Module

The most common approach - specify the module (tooth size) and gear ratio.

In [ ]:
# Design a worm gear set with module 2.0mm and 30:1 ratio
design = design_from_module(
    module=2.0,      # Module in mm (tooth size)
    ratio=30,        # Gear ratio (30:1)
    num_starts=1,    # Single-start worm
    pressure_angle=20.0,
    hand="right"
)

print(f"Worm pitch diameter: {design.worm.pitch_diameter_mm:.2f} mm")
print(f"Worm tip diameter: {design.worm.tip_diameter_mm:.2f} mm")
print(f"Wheel pitch diameter: {design.wheel.pitch_diameter_mm:.2f} mm")
print(f"Wheel tip diameter: {design.wheel.tip_diameter_mm:.2f} mm")
print(f"Centre distance: {design.assembly.centre_distance_mm:.2f} mm")
print(f"Lead angle: {design.worm.lead_angle_deg:.2f}°")

## Example 2: Design from Envelope (OD constraints)

When you have size constraints - specify maximum outer diameters.

In [ ]:
# Design to fit within specific outer diameters
design_envelope = design_from_envelope(
    worm_od=20.0,     # Maximum worm outer diameter
    wheel_od=65.0,    # Maximum wheel outer diameter  
    ratio=30,
    od_as_maximum=True,       # Treat ODs as maximum, find best fit
    use_standard_module=True  # Use standard module sizes
)

print(f"Module: {design_envelope.worm.module_mm:.3f} mm")
print(f"Worm OD: {design_envelope.worm.tip_diameter_mm:.2f} mm (max was 20.0)")
print(f"Wheel OD: {design_envelope.wheel.tip_diameter_mm:.2f} mm (max was 65.0)")

## Validation

Check the design for engineering issues.

In [ ]:
# Validate the design
validation = validate_design(design)

print(f"Valid: {validation.valid}")
print(f"\nErrors ({len(validation.errors)}):")
for error in validation.errors:
    print(f"  ❌ {error}")
    
print(f"\nWarnings ({len(validation.warnings)}):")
for warning in validation.warnings:
    print(f"  ⚠️ {warning}")

## Export to JSON

Export the design to JSON format. This can be used with:
- `wormgear-geometry` CLI tool (local install)
- [wormgear.studio](https://wormgear.studio) web interface

In [ ]:
# Convert to JSON
design_json = to_json(design)

# Pretty print
print(json.dumps(design_json, indent=2))

In [ ]:
# Save to file (can be downloaded from Colab)
with open('wormgear_design.json', 'w') as f:
    json.dump(design_json, f, indent=2)

print("Saved to wormgear_design.json")
print("\nTo generate STEP files locally:")
print("  pip install wormgear")
print("  wormgear-geometry wormgear_design.json")

In [ ]:
# Download the file (Colab-specific)
try:
    from google.colab import files
    files.download('wormgear_design.json')
except ImportError:
    print("Not running on Colab - file saved locally")

## All Design Parameters

Full list of available parameters for `design_from_module()`:

In [ ]:
# Full parameter example
design_full = design_from_module(
    module=2.0,              # Module (mm) - tooth size
    ratio=30,                # Gear ratio
    num_starts=1,            # Number of worm starts (1, 2, or 4 typical)
    pressure_angle=20.0,     # Pressure angle (degrees) - 20° standard
    worm_pitch_diameter=None, # Auto-calculated if None
    profile_shift=0.0,       # Profile shift coefficient
    backlash=0.05,           # Backlash (mm)
    hand="right",            # Thread hand: "right" or "left"
    worm_type="cylindrical", # "cylindrical" or "globoid"
    profile="ZA",            # "ZA" (straight) or "ZK" (convex)
)

print(f"Design complete: {design_full.worm.module_mm}mm module, {design_full.assembly.ratio}:1 ratio")